In [9]:
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from IPython.display import display, Markdown
from dotenv import load_dotenv
import time
import os
from pinecone import Pinecone 
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec


In [10]:
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key)
client = OpenAI()

In [11]:
index_name = "ia-sensor"

if not pc.has_index(index_name):
   pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

In [ ]:
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)


#Retriver
retriever = vectorstore.as_retriever()
# Pegar os documentos relevantes baseado na pergunta
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)

# Preparar o contexto
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)

# Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", 
         "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado, em formato de passo a passo.\n\n{context}"
         },
        {"role": "user", 
        "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
         } 
    ]
)

# Resposta
display(Markdown(completion.choices[0].message.content))
